# 1. Introduction

Indian Premiere League is a 20 over cricket (T20) tournament played in India every year. It generally comprises of 8 teams representing the 8 Indian cities and is one of the most watched cricket tournaments around the world. It is also one of the most cash rich leagues in cricket.

In this project I have analyzed the data of IPL matches from 2008 to 2019 to gather insights. The current edition of IPL 2020 has not been included in the analysis. The data has been collected from the Kaggle website (https://www.kaggle.com/nowke9/ipldata).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib'

# 2. Data Gathering

The data has been gathered from the Kaggle website and has been downloaded in a csv format into the local drive.

In [ ]:
# Reading the match data from the csv file
match = pd.read_csv(r'/kaggle/input/ipldata/matches.csv')
match.head(2)

In [ ]:
# Reading the data for each ball 
ball= pd.read_csv(r'/kaggle/input/ipldata/deliveries.csv')
ball.head(2)

# 3. Data Wrangling

### 3.1 Missing Values

In [ ]:
match.info()

So there are total 756 matches with 752 decided winners. Let us see which rows do not have any decided winner.

In [ ]:
match[match.isna()['winner']]

So these matches were probably washed from the rains and did not have any results declared.

In [ ]:
ball.info()

This dataframe has missing values only for player dismissed and dismissal_kind. <br>

The data type for all the columns also seem to be okay.

### 3.2 Duplicate values

In [ ]:
match.duplicated().sum()

In [ ]:
ball.duplicated().sum()

In [ ]:
ball[ball.duplicated()].head()

In [ ]:
ball.query('match_id==221 & inning==1 & over==4 & ball==1')

So these duplicate values need to be removed as they dont add any value.

In [ ]:
ball.drop_duplicates(inplace= True)

### 3.3 Renaming Franchise team names
Some teams like Deccan Chargers/ Sunrisers Hyderabad can be grouped together as they were the same teams.

In [ ]:
# Replacing Franchise names
ball.replace({'Sunrisers Hyderabad':'Hyderabad (Sunriser/Chargers)','Deccan Chargers':'Hyderabad (Sunriser/Chargers)',\
'Rising Pune Supergiants':'Pune (Supergiant/ Warriors)','Delhi Daredevils':'Delhi (Capitals/ Daredevils)',\
'Delhi Capitals':'Delhi (Capitals/ Daredevils)','Pune Warriors':'Pune (Supergiant/ Warriors)',
'Rising Pune Supergiant':'Pune (Supergiant/ Warriors)'}, inplace= True)

In [ ]:
ball.batting_team.unique()

In [ ]:
match.replace({'Sunrisers Hyderabad':'Hyderabad (Sunriser/Chargers)','Deccan Chargers':'Hyderabad (Sunriser/Chargers)',\
'Rising Pune Supergiants':'Pune (Supergiant/ Warriors)','Delhi Daredevils':'Delhi (Capitals/ Daredevils)',\
'Delhi Capitals':'Delhi (Capitals/ Daredevils)','Pune Warriors':'Pune (Supergiant/ Warriors)',
'Rising Pune Supergiant':'Pune (Supergiant/ Warriors)'}, inplace= True)

In [ ]:
match.team2.nunique()

### 3.4 Creating summary dataframes from ball dataframe

In [ ]:
# Creating a new df summarizing total runs and wickets that fell in each innings

summary= ball.groupby(by=['match_id','inning','batting_team']).sum()['total_runs'].reset_index()
temp = ball.groupby(by=['match_id','inning','batting_team']).count()['player_dismissed'].reset_index()

summary= pd.merge(summary, temp,how='outer', on= ['match_id','inning', 'batting_team'])

In [ ]:
# Creating a new column combining the over and the ball bowled
ball['over_ball']= ball['over']+ball['ball']/10

In [ ]:
# Merging the total balls used in the innings with the summary data
temp= ball.groupby(by=['match_id','inning','batting_team']).max(numeric_only=True)['over_ball'].reset_index()

summary= pd.merge(summary, temp,how='outer', on= ['match_id','inning', 'batting_team'])
summary.head(2)

In [ ]:
# Changing the matrix into a long format
s1 = summary.melt(id_vars=['match_id','inning',]).query('variable=="total_runs" & inning==1')[['match_id','value']]
s2 =summary.melt(id_vars=['match_id','inning',]).query('variable=="total_runs" & inning==2')[['match_id','value']]
temp= pd.merge(s1, s2, on='match_id', how='inner')

s1= summary.melt(id_vars=['match_id','inning',]).query('variable=="over_ball" & inning==1')[['match_id','value']]
s2=summary.melt(id_vars=['match_id','inning',]).query('variable=="over_ball" & inning==2')[['match_id','value']]
temp2= pd.merge(s1,s2,on='match_id', how='inner')

s1= summary.melt(id_vars=['match_id','inning',]).query('variable=="player_dismissed" & inning==1')[['match_id','value']]
s2=summary.melt(id_vars=['match_id','inning',]).query('variable=="player_dismissed" & inning==2')[['match_id','value']]
temp3 = pd.merge(s1,s2,on='match_id', how='inner')

summary = pd.merge(pd.merge(temp,temp2, on= 'match_id', how='inner'), temp3, on='match_id', how='inner')

summary.rename(columns={'value_x_x':'runs_1', 'value_y_x':'runs_2','value_x_y':'overs_1','value_y_y':'overs_2',\
    'value_x':'wickets_1', 'value_y':'wickets_2'}, inplace= True)
summary.head()

In [ ]:
# Merging the summary data with match info data
match= pd.merge(match, summary, right_on= 'match_id', left_on= 'id', how='left')

### 3.5 Droppping umpire information as it is not essential for our analysis

In [ ]:
match.drop(columns=['umpire1', 'umpire2', 'umpire3'], inplace= True)

In [ ]:
match.head(2)

### 3.6 Creating a batsman summary dataframe

In [ ]:
# Total runs scored by each batsman
batsman= ball.groupby(by='batsman').sum()['batsman_runs'].reset_index()

#Total balls played by each batsman
temp = ball.groupby(by='batsman').count()['ball']
batsman= pd.merge(batsman, temp, how='outer',on='batsman')

# Total 4s hit
temp = ball.query('batsman_runs==4').groupby(by='batsman').count()['batsman_runs'].reset_index()
batsman = pd.merge(batsman, temp, how='outer',on='batsman')

#Total 6s hit
temp= ball.query('batsman_runs==6').groupby(by='batsman').count()['batsman_runs'].reset_index()
batsman = pd.merge(batsman, temp, how='outer',on='batsman')

batsman.rename(columns={'batsman_runs_x':'total_runs','batsman_runs_y':'4s', 'batsman_runs':'6s'}, inplace= True)
batsman.head(3)

In [ ]:
# Calculating the strike rate
batsman['strike_rate']= (batsman.total_runs/ batsman.ball)*100
batsman.head(2)

In [ ]:
# Calculating percent of the runs scored by boundries
batsman['boundary_percent']= (batsman['4s']*4+ batsman['6s']*6)/ batsman.total_runs
batsman.head(1)

In [ ]:
# Calculating the runs scored by openers in each match
#calculating the runs scored by each batsman in each match
a= ball.groupby(by=['match_id','batsman']).sum()['batsman_runs']
b= ball.groupby(by=['match_id','batsman']).count()['total_runs']
opening = pd.DataFrame(data=[a,b]).transpose().reset_index()

# finding which batsmen opened
opening2= ball.query('over==1 & ball==1')[['match_id','batsman','non_striker']].melt(id_vars='match_id')
opening = pd.merge(opening, opening2, how='right',left_on =['match_id','batsman'], right_on=['match_id','value'])

#renaming columns
opening.rename(columns={'total_runs':'opening_balls', 'batsman_runs':'opening_runs'}, inplace= True)
#dropping redundant columns
opening.drop(columns=['variable','value'], inplace= True)
opening.head()

In [ ]:
# Aggregating the runs scored by openers
opening= opening.groupby(by='batsman').sum()[['opening_runs','opening_balls']].reset_index()

batsman= pd.merge(batsman, opening, on='batsman', how='left')
batsman.head(3)

### 3.7 Creating a bowler summary dataframe

In [ ]:
ball.dismissal_kind.value_counts()

For calculating the wickets picked by the bowler we need to exclude the run outs, retired hurt and obstructing the field. There are actually 2 dismissals for obstructing the field!!! <br>

A quick Google search reveals that they were probable runouts but the batsmen (Yusuf Pathan and Amit Mishra) blocked the ball from hitting the stumps and hence were declared out.

In [ ]:
ball.head(2)

In [ ]:
# Total balls bowled and total runs given by the bowler
bowler= ball.groupby(by='bowler').count()['ball'].reset_index()
temp= ball.groupby(by='bowler').sum()['total_runs']

bowler= pd.merge(bowler, temp, on='bowler', how='outer')

In [ ]:
# Filtering the runouts/ retired hurts
temp = ball.query('dismissal_kind!="obstructing the field" & dismissal_kind !="retired hurt" & dismissal_kind != "run out"')

# Filtering the wickets taken
temp = temp[~temp.isna().dismissal_kind][['bowler','dismissal_kind']]
temp= temp.groupby(by=['bowler']).count().reset_index()

bowler= pd.merge(bowler, temp, on='bowler', how='outer')

In [ ]:
# Renaming the column to reflect the wickets
bowler.rename(columns={'dismissal_kind':'wickets_taken'}, inplace= True)

In [ ]:
# Calculating the total dot balls
temp = ball.query('total_runs==0').groupby(by='bowler').count()['ball'].reset_index()
bowler= pd.merge(bowler, temp, on='bowler', how='outer')

In [ ]:
# Renaming the column to reflect the wickets
bowler.rename(columns={'ball_x':'balls_bowled', 'ball_y':'dot_balls'}, inplace= True)
bowler.head(1)

In [ ]:
# Calculating the number of bowled and LBWs taken by the bowler
temp =ball.query('dismissal_kind=="bowled" | dismissal_kind =="lbw" ').groupby(by='bowler').count()['dismissal_kind'].reset_index()
bowler= pd.merge(bowler, temp, on='bowler', how='outer')
bowler.head(3)

In [ ]:
# Renaming the column to reflect the wickets
bowler.rename(columns={'dismissal_kind':'bowl_or_lbw'}, inplace= True)

In [ ]:
bowler.head(1)

In [ ]:
# Calculating the bowler economy
bowler['economy']= bowler.total_runs/ (bowler.balls_bowled/6)

In [ ]:
# Calculating dot ball percent
bowler['dot_ball_percent']= bowler.dot_balls/bowler.balls_bowled

In [ ]:
# Calculating the runs scored in last 3 overs (death over) to compare the bowlers performance
a= ball.query('over>=18').groupby('bowler').sum()['total_runs']
b= ball.query('over>=18').groupby('bowler').count()['batsman_runs']
temp =pd.DataFrame(data=[a,b] ).transpose().reset_index()
temp.rename(columns={'total_runs':'death_runs','batsman_runs':'death_balls' }, inplace= True)

bowler= pd.merge(bowler, temp, on='bowler', how='outer')
bowler.head(3)

In [ ]:
# Calculating bowling economy in death over
bowler['death_econ']= bowler.death_runs/ (bowler.death_balls/6)

In [ ]:
# Calculating the percentage of bowled and LBWs by the bowler among the wickets they took
bowler['bowl_lbw_percent']= bowler.bowl_or_lbw/ bowler.wickets_taken

In [ ]:
bowler.head(1)

### 3.8 Team inning summary
To calculate over wise run scoring for each team

In [ ]:
# Total innings played for each over bowled
overs_played= ball[['batting_team','over','match_id']].drop_duplicates().groupby(by=['batting_team','over']).count()['match_id'].reset_index()

In [ ]:
# Total runs scored in each over
over_runs= ball.groupby(by=['batting_team','over']).sum()['total_runs'].reset_index()
over_runs= pd.merge(over_runs, overs_played, on=['batting_team','over'], how='outer')

over_runs.rename(columns={'match_id':'matches'}, inplace= True)

In [ ]:
# Calculating average runs scored in each over
over_runs['avg']= over_runs.total_runs/ over_runs.matches

In [ ]:
over_runs.head()

### 3.9 Changing the datatype for Match columns

In [ ]:
# Changing the data type for the match columns to float values
float_list=['runs_1', 'runs_2','overs_1', 'overs_2','wickets_1', 'wickets_2' ]
for col in float_list:
    match.loc[:,col] = match[col].astype(float)
match.info()

### 3.10 Replacing the city names that are same

In [ ]:
match.replace({'Bengaluru':'Bangalore'}, inplace= True)

### 3.11 Which team won the match (batting or fielding first)

In [ ]:
match['batting_first_won']= match.winner== match.team1

# 4. Exploratory Data Analysis

### 4.1. Average runs scored in each over

In [ ]:
# Plotting average runs scored in each over in IPL
series= over_runs.groupby(by='over').mean()['avg']
sns.set_style('darkgrid')

plt.figure(figsize=(10,6))
sns.barplot(x=series.index, y=series, color='tab:blue', )
plt.axhline(over_runs.avg.mean(), color='r', label= 'Avg runs scored total')
plt.title('Average runs scored in each over')
plt.ylabel('Average runs scored')
plt.xlabel('Over number')
plt.yticks(np.arange(0,11))
plt.legend()
plt.show()

To see trends of the average runs scored in each over I plotted a barplot between the over and the runs scored in that over. On an average teams score approximately 8 runs per over.

However, almost all teams have a cautious start to the innings scoring between 6-7 runs in the first 2 overs and accelerating in the next 4 overs before the powerplay ends. 

The slog overs 17+ see on an average 9-10 runs being scored per over.

### Average runs scored per over team wise

In [ ]:
color_code= {'Chennai Super Kings':'y','Delhi (Capitals/ Daredevils)':'b','Hyderabad (Sunriser/Chargers)':'tab:orange',\
  'Kings XI Punjab':'r', 'Kolkata Knight Riders':'k','Mumbai Indians':'tab:blue','Rajasthan Royals':'tab:pink',\
     'Royal Challengers Bangalore':'tab:red'}

In [ ]:
# Average runs scored per over team wise
sns.set_style('darkgrid')
plt.figure(figsize=(10,6))

temp= over_runs.query('batting_team !="Kochi Tuskers Kerala" & batting_team != "Pune (Supergiant/ Warriors)" & \
batting_team !="Gujarat Lions"')
sns.lineplot(x='over', y='avg', data=temp, hue='batting_team', palette=color_code)
plt.xticks(np.arange(1,21))
plt.legend(loc=(1.05,0.3))
plt.title('Average runs scored per over (team wise)')
plt.xlabel('Over')
plt.ylabel('Average runs scored')
plt.show()

Chennai Super Kings makes one of the slowest starts among all the IPL teams but finishes among the highest run scorers in the last few overs. It starts by scoring about 2-3 runs in the first over, compared to 7 runs scored by Kolkata Knight Riders (KKR).

However, CSK finished with about 11+ runs in the final over compared to only 9 runs by KKR.

The trends of runs scored among all the teams see similar peaks and valleys

### 4.2 Winning percent team and inning wise

In [ ]:
# Grouping the wins for teams batting first and second
temp = match.groupby(by=['winner','team1']).count()['id'].reset_index().query('winner==team1')[['winner','id']]
temp2 = match.groupby(by=['winner','team2']).count()['id'].reset_index().query('winner==team2')[['winner','id']]
temp3= pd.DataFrame([match.team1.value_counts(),match.team2.value_counts()],).transpose().reset_index()

team_stat= pd.merge(temp, temp2, on='winner', how='inner')
team_stat= pd.merge(team_stat, temp3, left_on='winner',right_on='index', how='inner')

team_stat.rename(columns={'id_x':'batting_first','id_y':'batting_second'}, inplace= True)
team_stat.drop(columns=['index'], inplace= True)

In [ ]:
# finding win percent inning wise
team_stat['win_percent_first']= team_stat['batting_first']/ team_stat['team1']
team_stat['win_percent_second']= team_stat['batting_second']/ team_stat['team2']

In [ ]:
# Aggregating the win percent
team_stat['win_percent']=(team_stat.batting_first+ team_stat.batting_second)/ (team_stat.team1+ team_stat.team2)

In [ ]:
# Sorting the values
team_stat.sort_values(by='win_percent', inplace=True, ascending=False)

In [ ]:
plt.figure(figsize=(8,6))
sns.set_style('darkgrid')

sns.lineplot(x= 'winner', y= 'win_percent_first',data=team_stat, color='g',label='Batting First', sort= False, marker='o' )
sns.lineplot(x= 'winner', y= 'win_percent_second', data=team_stat, color='y', label= 'Fielding First',sort=False,marker='o')
sns.barplot(x='winner', y= 'win_percent',data=team_stat,  color='tab:blue', label='Average winning rate')

plt.xticks(rotation=90)
plt.xlabel('IPL Team')
plt.ylabel('Winning Percent')
plt.title('Winning percent for each team')
plt.legend()
plt.show()

Unsurprisingly, CSK has one of the highest win percents in the IPL games winning about 61% of its games. However, it has a slightly better chasing record (65%) than a batting first record (57% wins). Interstingly, all the teams have a better chasing record than batting first record (chasing team wins 55% matches).

This difference is most prominent in Kolkata Knight Riders who have a win percent of 61% chasing but only 41% batting first. Similary, Rajasthan Royals also have a large gap between the chasing (57.5%) and batting first (43%) win percents. Mumbai Indians has almost similar winning percent for both batting and fielding first (57% and 59% respectively).

In [ ]:
team_stat

### Cliffhanger matches
We need to see how many matches were decided in the last over to see if they were cliffhanger matches. 

To see this we will find how many times Team 2 won the match after the 20.3th over and how many times Team 1 won by less than 6 runs (maximum runs that can be scored on a ball).

In [ ]:
# Finding matches where Team 1 won by less than 6 runs or Team 2 won in last 3 balls
cliff= match.query('(win_by_runs <6 & win_by_runs >0) | (overs_2 > 20.3 & win_by_wickets >0)')

In [ ]:
# Grouping the wins for teams batting first and second
temp = cliff.groupby(by=['winner','team1']).count()['id'].reset_index().query('winner==team1')[['winner','id']]
temp2 = cliff.groupby(by=['winner','team2']).count()['id'].reset_index().query('winner==team2')[['winner','id']]

In [ ]:
# Grouping the wins for teams batting first and second
temp = cliff.groupby(by=['winner','team1']).count()['id'].reset_index().query('winner==team1')[['winner','id']]
temp2 = cliff.groupby(by=['winner','team2']).count()['id'].reset_index().query('winner==team2')[['winner','id']]
temp3= pd.DataFrame([cliff.team1.value_counts(),cliff.team2.value_counts()],).transpose().reset_index()

team_stat2= pd.merge(temp, temp2, on='winner', how='inner')
team_stat2= pd.merge(team_stat2, temp3, left_on='winner',right_on='index', how='inner')

team_stat2.rename(columns={'id_x':'batting_first','id_y':'batting_second'}, inplace= True)
team_stat2.drop(columns=['index'], inplace= True)

# finding win percent inning wise
team_stat2['win_percent_first']= team_stat2['batting_first']/ team_stat2['team1']
team_stat2['win_percent_second']= team_stat2['batting_second']/ team_stat2['team2']

# Aggregating the win percent
team_stat2['win_percent']=(team_stat2.batting_first+ team_stat2.batting_second)/ (team_stat2.team1+ team_stat2.team2)

In [ ]:
team_stat2['total_wins']= team_stat2.batting_first + team_stat2.batting_second

# Sorting the values
team_stat2.sort_values(by='total_wins', inplace=True, ascending=False)
team_stat2.head()

In [ ]:
plt.figure(figsize=(8,6))
sns.set_style('ticks')

sns.barplot(team_stat2.winner, team_stat2.total_wins ,label='Total wins', color='tab:blue')
plt.xticks(rotation=90)
plt.xlabel('IPL Team')
plt.ylabel('Total wins')

axes2= plt.twinx()
axes2.plot(team_stat2.winner,team_stat2.win_percent_first ,'g-o',label='Batting First')
axes2.plot(team_stat2.winner,team_stat2.win_percent_second ,'y-o',label='Fielding First')
axes2.plot(team_stat2.winner,team_stat2.win_percent ,'r-o',label='Average winning rate')

plt.ylabel('Winning Percent')
plt.yticks(np.arange(0,1.1,0.1) )
plt.title('Cliffhanger wins for each team')
plt.legend(loc=9 )
plt.show()

Chennais Super Kings are again the kings of winning cliffhanger matches. Among the 24 matches where they were chasing that were decided in the last 3 balls, they won whooping 87% of those games (21 games). Rajasthan Royals won 80% of these types of games (12/15 matches). 

When defending a target Mumbai Indians seem to better at holding the nerves as they won 41% of the matches where the winning margin was less than 6 runs.

Overall CSK and RR have won the most cliffhanger games winning 64% and 65% of these games respectively.

In [ ]:
team_stat2

### Distribution of wins
We will next see the distribution of wins for the teams batting first and second.

In [ ]:
# Distribution for team batting first
temp= match.query('win_by_runs!=0')

plt.figure(figsize=(8,6))
plt.hist(temp.win_by_runs, bins=15)
plt.xticks(np.arange(0,160,10))
plt.xlabel('Margin of win by runs')
plt.ylabel('Frequency')
plt.title('Distribution for victory margin(in runs)')
plt.show()

In [ ]:
match.query('batting_first_won==False').shape[0]

In [ ]:
160/421

Overall, the team batting first won 44.3% of the games while the team batting second won 55.7% of these games.

As the histogram above shows about 24% of the games that were won by the batting first team, the winning margin was less than 10 runs. 

However, 38% of the games won by the winning team were won in the 20th over, showing how close the matches have been.

In [ ]:
# Distribution for team batting second
temp = match.query('win_by_wickets>0').overs_2.apply(lambda x: int(x))

plt.figure(figsize=(8,6))
plt.hist(temp, bins= 16  )
plt.xticks(np.arange(5,21,1))
plt.xlabel('Overs')
plt.ylabel('Frequency')
plt.title('Which over did the chasing team win the match?')
plt.show()

### Best Batting statistics

In [ ]:
# Most runs scored by a batsman
temp=batsman.sort_values(by='total_runs', ascending= False).head(7)
sns.set_style('dark')

plt.figure(figsize= (8,6))
sns.barplot(x= temp.batsman, y= temp.total_runs, color= 'tab:blue')
plt.ylabel('Total Runs scored')
plt.xlabel('Batsmen')
plt.title('Most runs scored by a batsman')
plt.yticks(np.arange(0,5600,500))

axes2= plt.twinx()
axes2.plot(temp.batsman, temp.boundary_percent*100, 'r-o', label='Percent of runs scored in boundary')
axes2.plot(temp.batsman, temp.strike_rate, 'y-o', label='Strike Rate')
plt.ylabel('Percent/ Strike Rate')
plt.yticks(np.arange(50,180,10))
plt.legend()
plt.show()

Virat Kohli and Suresh Raina are among the highest run scorers in IPL with each scoring more than 5400 runs. However, one of the highest strike rates (145%) among the top batsmen is by Chris Gayle. 

Gayle also has one of the highest number of runs scored in boundaries with a whooping 76% runs being scored in 4s and 6s. David Warner comes next with about 61% runs being scored in boundaries among the top 7 batsmen.

In [ ]:
# Most runs scored by an opening batsman
temp=batsman.sort_values(by='opening_runs', ascending= False).head(7)
sns.set_style('dark', )

plt.figure(figsize= (8,6))
sns.barplot(x= temp.batsman, y= temp.opening_runs, color= 'tab:blue')
plt.ylabel('Total Runs scored')
plt.xlabel('Batsmen')
plt.title('Most runs scored by an opener')
plt.yticks(np.arange(0,5100,500))

axes2= plt.twinx()
axes2.plot(temp.batsman, temp.boundary_percent*100, 'r-o', label='Percent of runs scored in boundary')
axes2.plot(temp.batsman, temp.strike_rate, 'y-o', label='Strike Rate')
plt.ylabel('Percent/ Strike Rate')
plt.yticks(np.arange(50,180,10))

plt.legend()
plt.show()

Chris Gayle, Shikhar Dhawan and David Warner are among the highest run scorers for openers having scored more than 4000 runs in the opening position.

Interestingly, Sehwag and McCullum who are also among other top openers, like Gayle, they have also scored 72% and 67% of their runs in boundaries.

### Best Bowling Statistics

In [ ]:
# Most wickets taken
temp= bowler.sort_values(by='wickets_taken', ascending= False).head(7)

plt.figure(figsize=(9,6))
sns.barplot(x= temp.bowler, y= temp.wickets_taken, color= 'tab:blue')
plt.ylabel('Wickets taken')
plt.yticks(np.arange(0,190,20))

axes2= plt.twinx()
axes2.plot(temp.bowler, temp.bowl_lbw_percent, 'g-o', label='Percent of bowled/LBW dismissals')
axes2.plot(temp.bowler, temp.dot_ball_percent, 'r-o', label='Percent dot balls')
plt.ylabel('Percent', )
plt.yticks(np.arange(0,0.6,0.1))
plt.legend()
plt.title('Most wickets taken and their economy')

plt.show()

In [ ]:
temp

S. Malinga, Amit Mishra and Harbhanjan Singh are one of the highest wicket takers in IPL. 43% of the wickets taken by Malinga have been either bowled or LBW showing how effective bowler he is. Piyush Chawla also has a similar bowled/lbw dismisal percent (39%).

Bhuvaneshwar Kumar has one of the highest dot ball percents (41%) among the leading wicket takers slightly ahead of Malinga (38%).

In [ ]:
# Bowlers with best economy
temp =bowler.query('balls_bowled > 120').sort_values(by='economy', ascending= True).head(10)
sns.set_style('darkgrid')
plt.figure(figsize=(12,6))
sns.lineplot(temp.bowler, temp.economy, color='tab:red', marker='o', label='Economy', sort= False)
sns.lineplot(temp.bowler, temp.death_econ, color='tab:green', marker='o', label='Economy in death overs (>18)', sort= False)
sns.lineplot(temp.bowler, temp.dot_ball_percent*6, color='tab:blue', marker='o', label='Average dot balls per over', sort= False)
plt.yticks(np.arange(0,11,1))
plt.ylabel('Economy/ Dot balls per over')
plt.xlabel('Bowler')
plt.title('Most economical bowlers')
plt.legend()
plt.show()

In [ ]:
bowler.query('balls_bowled > 120').sort_values(by='death_balls', ascending= False)

Among major bowlers (who bowled more than 20 overs), Sohail Tanvir and A Chandillan are one of the most economical bowlers with an economy of around 6.22 and 6.28. However, Anil Kumble has one of the best economies in the death overs of 6.64 compared to 7.8 of Sohail Tanvir. (I took the last 3 overs of the innings as the death overs.)

Bravo and Malinga seem to have the most trust of their captain in death overs, having bowled a total of 130 and 120 overs respectively in the death overs of the matches.

### Venue Analysis

In [ ]:
# Win percentage for team batting first on each venue
major_city= ['Bangalore','Chandigarh','Chennai','Delhi','Hyderabad','Jaipur','Kolkata','Mumbai','Pune']

temp = match[match['city'].isin(major_city) ]

In [ ]:
# Win percentage for team batting first on each venue
temp2 = temp.groupby(by='city').batting_first_won.mean()

plt.figure(figsize=(9,6))
sns.barplot(x=temp2.index, y=temp2, color='tab:blue')
plt.axhline(match.batting_first_won.mean(), color='r', label='Average winning rate for team batting first', linestyle='--')
plt.xlabel('Venue for the match')
plt.ylabel('Percent win for team batting first')
plt.title('Winning percent for team batting first across the venues')
plt.yticks(np.arange(0,0.7,0.05))
plt.legend()
plt.show()

As we saw earlier, the team batting first has a winning percent of 44.3%. However, across the stadiums in Chennai the team batting first wins whooping 61% of the games while this number is very low in Jaipur (32%). This could probably be explained from the fact that CSK which has a higher win percent among all teams mostly bats first when playing in Chennai.

Bangalore and Mumbai are venues that have higher than average runs scored (median of 170 runs) compared to Hyderabad which has lower scoring matches (median around 155).

In [ ]:
major_teams= ['Hyderabad (Sunriser/Chargers)', 'Kolkata Knight Riders', 'Kings XI Punjab',
       'Royal Challengers Bangalore', 'Mumbai Indians',
       'Delhi (Capitals/ Daredevils)','Chennai Super Kings', 'Rajasthan Royals']
temp.pivot_table(index='city', columns='winner',values='batting_first_won', aggfunc='mean' )[major_teams]

In [ ]:
plt.figure(figsize=(8,6))

sns.boxplot(x=temp.city, y=temp.runs_1,color= 'tab:blue' )
plt.axhline(match.runs_1.median(), color='r', linestyle='--')
plt.yticks(np.arange(0,325,20))
plt.ylabel('Runs scored on the venue')
plt.xlabel('Venue')
plt.title('Runs scored on each venue')
plt.show()

In [ ]:
# Venue wise distribution for runs scored by team batting first
plt.figure(figsize=(9,6))
temp= match[match['city'].isin(major_city)]
sns.scatterplot(x= temp.runs_1, y= temp.city, hue= temp.batting_first_won, y_jitter= 1 )
plt.xticks(np.arange(60,270,20))
plt.legend(loc=(1.05,0.5))
plt.xlabel('Runs scored by team batting first')
plt.ylabel('Venue of the match')
plt.title('Runs scored by team batting first in each venue')
plt.show()

In [ ]:
# Venue wise distribution for runs scored by team batting first
plt.figure(figsize=(9,6))
#temp= match[match['team1'].isin(major_teams)]
sns.scatterplot(x= match.runs_1, y= match.team1, hue= match.batting_first_won, y_jitter= 1 )
plt.xticks(np.arange(60,270,20))
plt.legend(loc=(1.03,0.8))
plt.xlabel('Runs scored by team batting first')
plt.ylabel('IPL Teams')
plt.title('Runs scored by team batting first vs winning record')
plt.show()

### Most runs scored in winning cause

In [ ]:
# Merging the match winner to the Ball dataframe
ball= pd.merge(ball, match.loc[:,['id','winner']], left_on='match_id', right_on= 'id', how='left')

In [ ]:
temp = ball[ball.batting_team== ball.winner].groupby(by=['batsman', 'match_id']).sum()['batsman_runs'].reset_index()
s1= temp.groupby(by='batsman').sum()['batsman_runs'].sort_values(ascending= False).head(15)

temp1= ball.groupby(by=['batting_team','match_id', 'batsman']).sum()['batsman_runs'].reset_index()
temp2= pd.merge(temp1, match[['id','winner']], left_on= 'match_id', right_on='id', how='left')
temp2['is_win'] = temp2.winner== temp2.batting_team
temp3= temp2.query('batsman_runs>50').groupby(by='batsman').mean()['is_win'].sort_values(ascending= False).reset_index()
temp4= pd.merge(temp3[temp3.batsman.isin(s1.index)], s1.reset_index(),on='batsman', how='inner').sort_values(by='batsman_runs', ascending= False)

In [ ]:
sns.set_style('dark')
plt.figure(figsize=(8,6))
sns.barplot(y= 'batsman_runs', x= 'batsman', data= temp4, color= 'tab:blue')
plt.xticks(rotation=90)
plt.title('Most runs scored in winning cause and win percent')
#plt.yticks('Runs scored by batsmen in winning cause')

axes= plt.twinx()
axes.plot(temp4.batsman, temp4.is_win,  'r--o', label='Win percent when batsman scored >50 runs')
plt.ylabel('Win percent when batsman scored more than 50 runs')
plt.yticks(np.arange(0.4,0.90,0.05))

plt.legend()
plt.show()

Suresh Raina, Rohit Sharma and David Warner have scored the most number of runs in winning causes. In fact, whenever Raina scored more than 50 runs, 66% of the times his team ended up winning the match.

However, whenever Shane Watson scored more than 50 runs, his team ended up winning more than 81% of the times. This win percent is 74% for MS Dhoni whenever he scored half century.

### Most Man of the Match awards

In [ ]:
s= match.player_of_match.value_counts().head(10)
plt.figure(figsize=(8,6))
sns.barplot(y= s.index, x= s, color= 'tab:blue')
plt.xlabel('Number of awards')
plt.ylabel('Player name')
plt.title('Most number of Player of the Match awards')
plt.xticks(np.arange(0,24,2))
plt.show()

Chris Gayle and Ab de Villiers have won the most number of Player of the Match awards.

In [ ]:
temp= ball.groupby(by=['batsman', 'match_id']).sum()['batsman_runs'].reset_index()

temp['>25']= temp.batsman_runs> 25
s1= temp.groupby(by='batsman').mean()['>25'].sort_values(ascending= False)
s2= temp.groupby(by='batsman').count()['match_id'].sort_values(ascending= False)

temp['>50']= temp.batsman_runs> 50
s3= temp.groupby(by='batsman').mean()['>50'].sort_values(ascending= False)

df1= pd.DataFrame([s1,s3,s2]).transpose()
temp2= df1.query('match_id>=10').head(10)

plt.figure(figsize=(8,6))
sns.barplot(x= temp2.index, y= temp2.match_id, color= 'tab:blue')
plt.xticks(rotation=90)
plt.ylabel('Total matches played')
plt.title('Batsman consistency across matches')

axes= plt.twinx()
axes.plot(temp2.index, temp2['>25'], 'g--o', label='Percent when batsman scored > 25')
axes.plot(temp2.index, temp2['>50'], 'r--o', label='Percent when batsman scored > 50')
plt.legend()
plt.xticks(rotation=90)
plt.xlabel('Player name')
plt.ylabel('Percent')
plt.show()

To measure the consistency of batsmen I saw how many times they scored more than 25 runs and 50 runs across all the matches they played (should have played atleast 10 matches). 

J. Bairstow is the most consistent batsman who has scored atleast 25 runs in 70% of the matches he played. However, considering that Bairstow has played only 10 IPL matches, David Warner's record looks very impressive having scored more than 50 runs in 38% of the matches (and 25+ runs in 56% matches) and having played more than 120 matches.

In [ ]:
temp2

# 5. Summary

It was really fun analyzing the IPL data and there feels that there is so much more that can be done. The key takeaways from my analysis are-

**Team Statistics** <br>
a) CSK has one of the highest win percents in the IPL games winning about 61% of its games followed by Mumbai Indians. All the teams have a better chasing record than batting first record (chasing team wins 55.7% matches).This difference is most prominent in Kolkata Knight Riders who have a win percent of 61% chasing but only 41% batting first.

b) About 24% of the games that were won by the batting first team, the winning margin was less than 10 runs. However, 38% of the games won by the winning team were won in the 20th over, showing how close the matches have been.

**Batsman Statistics** <br>
a) Virat Kohli and Suresh Raina are among the highest run scorers in IPL with each scoring more than 5400 runs. However, Chris Gayle also has one of the highest number of runs scored in boundaries with a whooping 76% runs being scored in 4s and 6s. 

b) Suresh Raina, Rohit Sharma and David Warner have scored the most number of runs in winning causes. In fact, whenever Raina scored more than 50 runs, 66% of the times his team ended up winning the match (it is 81% for Shane Watson and 74% for MS Dhoni).

c) J. Bairstow is the most consistent batsman who has scored atleast 25 runs in 70% of the matches he played. However, David Warner's record looks very impressive having scored more than 50 runs in 38% of the matches (and 25+ runs in 56% matches) and having played more than 120 matches.

**Bowler Statistics** <br>
a) Lasith Malinga  and Amit Mishra are one of the highest wicket takers in IPL. 43% of the wickets taken by Malinga have been either bowled or LBW. Bhuvaneshwar Kumar has one of the highest dot ball percents (41%) among the leading wicket takers.

b) Among major bowlers (who bowled more than 20 overs), Sohail Tanvir and A Chandillan are one of the most economical bowlers with an economy of around 6.22 and 6.28. However, Anil Kumble has one of the best economies in the death overs of 6.64. 

**Stadium Statistics** <br>
a) The team batting first has a winning chance of 44.3%. However, in Chennai the team batting first wins whooping 61% of the games while this number is very low in Jaipur (32%).